# 모둘화됨

In [2]:
from accounts import jp
# jp.ID, jp.PW

In [34]:
import os
from datetime import datetime
import time
import requests
import re

import pandas as pd

from bs4 import BeautifulSoup

import selenium

from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [28]:
############################
# 크롬으로 잡플래닛 사이트 열기
############################

service = Service(executable_path="chromedriver.exe")
browser = webdriver.Chrome(service=service)

url = r"https://www.jobplanet.co.kr/job"

browser.maximize_window()
print(browser.get_window_size())

browser.get(r"https://www.jobplanet.co.kr/job")

{'width': 1512, 'height': 944}


In [29]:
############################
# 잡플래닛에 로그인 하기
############################

# 로그인 버튼 찾기 및 클릭
login_btn = browser.find_element(By.CSS_SELECTOR,"a.btn_txt.login")
login_btn.click()

# 로그인 ID와 PW 쳐야할 곳 클릭 + 적기
login_Id = browser.find_element(By.CSS_SELECTOR, "#user_email")
login_Id.click()
login_Id.send_keys(jp.ID)

login_Pw = browser.find_element(By.CSS_SELECTOR, "#user_password")
login_Pw.click()
login_Pw.send_keys(jp.PW)

# 이메일로 로그인 버튼 누르기
em_log = browser.find_element(By.CSS_SELECTOR, "fieldset > button")
em_log.click()

In [24]:
# 모둘화됨

In [30]:
###########################
# 잡플래닛 검색창에 회사 입력하고 맨 처음에 나오는 기업 클릭하기.
###########################
# 검색창 누르기
search_bar = browser.find_element(By.CSS_SELECTOR, "#search_bar_search_query")
search_bar.click()

# 내가 원하는 회사 입력하기
want_cp = input()
search_bar.send_keys(want_cp)
search_bar.send_keys(Keys.RETURN)

# 검색결과 기다리기.
time.sleep(1)

# 맨처음에 나오는 기업 클릭하기
company = browser.find_element(By.CSS_SELECTOR, "div.is_company_card > div:nth-child(1) > a")
company.click()

In [31]:
###########################
# 회사페이지에서 리뷰 버튼 누르기
###########################
# 팝업창이 있을 때는 팝업창 버튼을 누르고 리뷰버튼을 누르기
try:
    pop = browser.find_element(By.CSS_SELECTOR, "div.premium_modal_header > button")
    pop.click()
    review_bar = browser.find_element(By.CSS_SELECTOR, "li.viewReviews > a")
    review_bar.click()

except NoSuchElementException:
    # 만약 팝업창 버튼이 없다면 리뷰 버튼만 누르기
    try:
        review_bar = browser.find_element(By.CSS_SELECTOR, "li.viewReviews > a")
        review_bar.click()
        time.sleep(1)
        pop = browser.find_element(By.CSS_SELECTOR, "div.premium_modal_header > button")
        pop.click()
    except NoSuchElementException:
        pass
    pass

In [32]:
###########################
# 회사페이지에서 리뷰 가져오기.
###########################
def get_jp_reviews():
    """
    잡플래닛 회사 리뷰 상세 내용을 url을 받아서 내용을 반환.
    연결
    """
    # 리뷰들을 리스트로 저장하는 변수
    good_review_list = []
    bad_review_list = []

    # 이전페이지 변수
    pre_page = 0

    # 무한 루프를 돌기.
    while True:

        # 소스 끌어오기.
        html = browser.page_source
        soup = BeautifulSoup(html, "lxml")

        # 현재 페이지의 숫자를 가져오기.
        crt_btn = soup.select_one('strong.txtlink_page')
        crt_page = int(crt_btn.text.strip())

        # 다음버튼을 눌렀은데도 같은 페이지인 경우
        if crt_page == pre_page:
            break

        # html 내용 가져오기
        good_review = soup.select("section > div > div.ctbody_col2 > div > dl > dd:nth-child(2) > span")
        bad_review = soup.select("section > div > div.ctbody_col2 > div > dl > dd:nth-child(4) > span")

        # 텍스트로 추출한 내용을 리스트에 추가해서 넣어주기 위해 이용.
        for good_text in good_review: # for문은 for 채워져있는 값의 원소 in 채워져있는 리스트
            # 데이터 전처리
            good_text_clean_1 = good_text.get_text().strip()
            good_text_clean_2 = good_text_clean_1.replace("\n", " ")
            
            # 장점 리뷰에 넣기.
            good_review_list.append(good_text_clean_2)

        for bad_text in bad_review:
            # 데이터 전처리
            bad_text_clean_1 = bad_text.get_text().strip()
            bad_text_clean_2 = bad_text_clean_1.replace("\n", " ")
            
            # 단점 리뷰에 넣기.
            bad_review_list.append(bad_text_clean_2)

        time.sleep(1)

        # 다음 페이지로 이동하기 위한 버튼을 찾기.
        try:
            # 이전페이지의 번호를 갱신하기.
            pre_page = crt_page

            # 다음 버튼 찾기
            next_btn = browser.find_element(By.CSS_SELECTOR,f"a.btn_pgnext")

            # 버튼이 있으면 클릭합니다.
            next_btn.click()

            # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
            time.sleep(3)

        except:
            break

        # local에 디렉토리를 생성.
        save_path = f"/Users/gyeonmunju/Desktop/PlayData/jp_company"
        os.makedirs(save_path, exist_ok=True)

        # 데이터프레임으로 관리하겠다 지정. (데이터 생성하기.)
        good_df = pd.DataFrame({"긍정적 리뷰" : good_review_list})
        bad_df = pd.DataFrame({"부정적 리뷰" : bad_review_list})
        # 긍정적/부정적 리뷰 합치기
        jp_df = good_df.join(bad_df)

        # csv 파일로 저장.
        file_name = f"{want_cp}reviews.csv"
        save_file_path = os.path.join(save_path, file_name)
        jp_df.to_csv(save_file_path, index=False, encoding = "utf-8")

    return jp_df

# 함수 호출 후 결과 출력
get_jp_reviews()

# 브라우저 끄기
browser.close()

In [81]:
df1 = pd.read_csv("/Users/gyeonmunju/Desktop/PlayData/jp_company/이그니스reviews.csv")
df1

,jp_review
0,너무 급성장 했는지 아직까지도 갖춰진 체계는 없는데 뭔가를 많이 하려고 하는 회사.
1,모든게 금방 바뀌는 곳 한가지가 오래 가는 일이 없음
2,빠른 성장이 이루어지는 곳. 그러나 이를 감당할 리더의 부재.
3,브랜드 인지도 높고 희소성이 있어 대부분의 사람들이 알고 있음. 단백질 시장에서 자...
4,자율적인 분위기에 고속 성장을 이뤄나가고 있는 식품 스타트업
...,...
70,알바당시엔 굉장히 괜찮은 곳이었어요 직원분들도 다 좋아보였구요
71,자기만의 시간을 가지고 싶거나 워라밸이 중요하다면 추천하지않아요
72,겉보기와는 달리 매우 보수적인 회사. 야근을 강요하는 분위기. 그렇다고 효율적인 야...
73,랩노쉬 이미지같은 곳을 기대했다면 실망할것 투자나 외부의견에 영향을 많이 받는다


In [6]:
from jobplanet import get_jp_reviews
get_jp_reviews("제너레잇")

{'width': 800, 'height': 600}


,긍정적 리뷰,부정적 리뷰
0,스타트업임에도 불구하고 야근이 거의 없는 편 능력있고 좋은 동료,능력있고 좋은 동료들이지만 지인으로서 좋겠지만 직장 동료로서는 단점이다. 사업의 ...
1,"20-30대로 이루어져 수평적이고 젊은 분위기이며, 서비스의 성장을 위해 모두 함...",건물 시설이 노후한 편이다. 스타트업 특성상 체계가 잡히지 않은 부분들이 있으나 만...
2,"성실함, 노력을 바탕으로 자신의 능력을 증명하면 회사가 많은 기회를 줄 수 있음. ...",마켓핏을 찾기위해 잦은 사업방향 전환이 업무의 피로도를 높일 수 있음. 사무실이 위...
3,사람간 스트레스가 거의 없는 편임. 연차를 눈치 안보고 편하게 쓸 수 있음. 호칭을...,"사무실 위치가 몽촌토성역, 잠실역 근처에 있어 접근성은 좋으나 건물이 노후화 되었음"
4,회사에 직접적인 여향을 미치는 책임감 있는 역할을 맡을 수 있다. 팀원들도 항상 ...,성장하는 스타트업이기 때문에 쳬계적이고 조직적으로 운영되는 것이 조금 어려울 때도 ...


# 테스트

In [11]:
from jobplanet import get_jp_reviews
comp = '넥슨코리아'

In [12]:
get_jp_reviews(comp)

{'width': 800, 'height': 600}


,긍정적 리뷰,부정적 리뷰
0,"자유로운 분위기, 하고 싶은걸 할 수 있음. 필요한 복지는 모두 갖추고 있음. 노조...",대기업다운 업무프로세스 불필요 체계가 많고 탑다운이 강함. 동종 규모의 타 아이티회...
1,도토리 어린이집 복지가 좋습니다 기술력있는 사람들이 많다,연봉이 적다는게 단점 자율성도 조금 떨어지는편이다.그래도 나쁘지는 않다.
2,"연차에대한 눈치를 주지 않았고, 인턴이다보니 칼퇴가 보장되었다. 팀 내 분위기가 좋...","팀 업무 상, 컨퍼런스를 개최할 일이 많아서, 나의 업무와 관련된 일만 하지는 않았..."
3,"비포괄, 업무 체계가 잘 갖춰져있어 워라밸을 보장받을 수 있다. (팀바팀) 비교적 ...",일부 비협조적인 팀원들로 인해 분위기에 물을 흐림. 딱히 개선 여지가 없어보임.
4,"초과근무시 오버된 근무시간으로 자유롭게 사용하는 Off. 저렴한 사내식당과 까페, ...",이것도 부서 by 부서겠지만.. 업무량이 많고 야근이 잦습니다.
...,...,...
60,"자기개발비, 사내 식당, 커피, 운동시설등등 복지가 좋았습니다.","팀별로 다른 사내분위기, 역에서 조금 먼 회사위치, 사내 식당 외 다른 식당 가기 ..."
61,"조직마다 분위기가 천지차이긴 하나, 나름 수평적인 문화",규모에 맞게 다양한 복지제도가 있지만 딱히 내세울 건 없음
62,"게임을 좋아하는 사람에게 강추한다, 게임과 관련해서 다양하게 정말 재밌는 걸 많이 ...","대기업 특유의 조직적 정체기로 향하고 있었는데, 조직 별로 노력의 정도가 차이 나서..."
63,"수평적인 분위기, 자유로운 연차, 다양한 인종 경험, 쾌적한 사무실, 적당한 업무량",채용된 직무와 무관한 업무를 많이 지시 전공을 보고 그 직무에 합격을 했는데 특히 ...
